In [29]:
import torch
from egg.core import Interaction
import os
from glob import glob
from collections import Counter
from pprint import pprint


In [12]:
root_dir = '/home/dominik/Nextcloud/020_Masterstudium/Language Technology/LT2402_Master Thesis/experiments/runs/language-games/'
model_dir = os.path.join(root_dir, 'discriminator/')
dataset_dir = os.path.join(model_dir, 'dale-2/')
run_dir = os.path.join(dataset_dir, '2023-08-10_09-34-38_discriminator/')
train_interaction_path = glob(os.path.join(run_dir, "interactions/", "train/", "epoch*/", "interaction*"))[0]
test_interaction_path = glob(os.path.join(run_dir, 'interactions/', 'validation/', 'epoch*/', 'interaction*'))[0]

In [31]:
train_interaction: Interaction = torch.load(train_interaction_path)
test_interaction: Interaction = torch.load(test_interaction_path)

def remove_eos(tensor):
    for index, symbol in enumerate(tensor):
        if int(symbol) == 0:
            return tuple(tensor[:index].tolist())
        
def get_messages(interaction):
    messages = [remove_eos(message.max(dim=1).indices) for message in interaction.message]
    return messages, Counter(messages)

train_messages, train_counter = get_messages(train_interaction)
test_messages, test_counter = get_messages(test_interaction)

print(train_messages)
print(train_counter)

print(test_messages)
print(test_counter)

print(train_interaction)
train_image_mapping = {}
for key in train_counter:
    train_image_mapping[key] = [image_id for image_id, message in zip(train_interaction.aux_input['image_id'], train_messages)]
pprint(train_image_mapping)

[(7,), (1,), (1,), (3,), (5,), (5,), (1,), (3,), (7,), (3,), (3,), (4,), (1,), (3,), (5,), (8,), (5,), (4,), (7,), (3,), (1,), (3,), (1,), (6,), (5,), (9,), (3,), (3,), (5,), (8,), (8,), (3,), (), (8,), (2,), (6,), (4,), (3,), (9,), (7,), (), (1,), (1,), (3,), (3,), (5,), (1,), (9,), (5,), (6,), (7,), (5,), (1,), (3,), (3,), (5,), (2,), (6,), (8,), (6,), (6,), (8,), (5,), (6,), (5,), (3,), (4,), (5,), (6,), (6,), (8,), (9,), (1,), (1,), (2,), (6,), (7,), (), (2,), (6,), (4,), (1,), (7,), (), (4,), (7,), (8,), (6,), (1,), (8,), (2,), (8,), (8,), (3,), (1,), (6,), (6,), (1,), (2,), (2,), (3,), (3,), (5,), (4,), (), (2,), (6,), (8,), (5,), (2,), (3,), (4,), (), (9,), (7,), (4,), (9,), (1,), (6,), (9,), (5,), (1,), (6,), (), (4,), (5,), (), (3,), (9,), (5,), (1,), (6,), (7,), (3,), (7,), (6,), (9,), (6,), (7,), (7,), (2,), (), (5,), (8,), (3,), (), (6,), (3,), (3,), (6,), (2,), (8,), (3,), (2,), (6,), (7,), (6,), (1,), (3,), (), (2,), (6,), (2,), (6,), (3,), (4,), (4,), (7,), (3,), (2,), (

TypeError: 'NoneType' object is not subscriptable